In [1]:
!pip install simpletransformers pandas scikit-learn torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [4]:
# Load dataset
df = pd.read_csv("/content/filtered_sentiment_dataset.csv")

In [5]:
# Map sentiment labels to numerical values
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
reverse_label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
df["sentiment"] = df["sentiment"].map(label_map)


In [6]:
# Drop NaN values
df.dropna(inplace=True)

In [7]:
# Split dataset into train (80%), validation (10%), and test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["sentiment"])
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["sentiment"])

In [11]:
# Rename columns to match simpletransformers expectations
train_df = train_df.rename(columns={"text": "text", "sentiment": "labels"})
valid_df = valid_df.rename(columns={"text": "text", "sentiment": "labels"})
test_df = test_df.rename(columns={"text": "text", "sentiment": "labels"})

In [12]:
# Convert labels to integer type
train_df["labels"] = train_df["labels"].astype(int)
valid_df["labels"] = valid_df["labels"].astype(int)
test_df["labels"] = test_df["labels"].astype(int)


In [13]:
# Define model arguments
model_args = ClassificationArgs()
model_args.num_train_epochs = 7
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.learning_rate = 5e-6
model_args.weight_decay = 0.01
model_args.overwrite_output_dir = True
model_args.save_best_model = True
model_args.evaluate_during_training = True
model_args.use_early_stopping = True
model_args.early_stopping_patience = 2
model_args.output_dir = "xlm-roberta-sentiment-model"
model_args.best_model_dir = "xlm-roberta-best-model"

In [14]:
# Initialize the model
model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base",
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [15]:
# Train the model
model.train_model(train_df, eval_df=valid_df)

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 7:   0%|          | 0/630 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(4410,
 defaultdict(list,
             {'global_step': [630,
               1260,
               1890,
               2000,
               2520,
               3150,
               3780,
               4000,
               4410],
              'train_loss': [0.8716583251953125,
               0.617340087890625,
               0.38429513573646545,
               0.23548507690429688,
               0.18587493896484375,
               0.4386393129825592,
               0.113037109375,
               0.1146087646484375,
               0.2571563720703125],
              'mcc': [0.0,
               0.24793396104372475,
               0.330903457826062,
               0.35852521594309406,
               0.32216612778657283,
               0.2884870811392113,
               0.36945968597239875,
               0.3434913024955744,
               0.3683016346274413],
              'eval_loss': [0.43737960352173333,
               0.3975611257402203,
               0.38539276432387437,
           

In [16]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=accuracy_score)
print(f"Model Accuracy: {result['acc']:.4f}")

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Model Accuracy: 0.8460


In [17]:
# Function to predict sentiment
def predict_sentiment(text, model):
    predictions, raw_outputs = model.predict([text])
    confidence = torch.softmax(torch.tensor(raw_outputs[0]), dim=0).tolist()
    sentiment = reverse_label_map[predictions[0]]
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\nConfidence Scores: {confidence}\n")
    return sentiment

In [18]:
# Predict on test set
for text in test_df["text"].tolist()[:10]:
    predict_sentiment(text, model)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Text:  Thala  a sami  I am wit
Predicted Sentiment: Positive
Confidence Scores: [0.012395127434668616, 0.0002900147467142518, 0.987314857818617]



0it [00:00, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Text:  Ivan comedian aatchay epde hero Anaaan comedy hero subject tan sari Sivakarthigeyn kku
Predicted Sentiment: Negative
Confidence Scores: [0.7473439783996997, 0.0013473856831231154, 0.2513086359171771]



0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Text:  Thalaivernna naama super Strdahan pinnittaru trailere keddha irukkunna padam vandha vera leval
Predicted Sentiment: Positive
Confidence Scores: [0.2219405685939567, 0.00037884154893590063, 0.7776805898571074]

